<a href="https://colab.research.google.com/github/hukgithub/Voice-to-Voice_Chatbot/blob/main/V2V_CB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary libraries
!pip install openai-whisper groq gtts gradio

In [3]:
# Import libraries
import whisper
import os
from gtts import gTTS
import gradio as gr
from groq import Groq

In [7]:
os.environ["GROQ_API_KEY"] = "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"

In [9]:
# Load Whisper model for transcription
model = whisper.load_model("base")

# Set up Groq API client (ensure GROQ_API_KEY is set in your environment)
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


In [10]:
# Function to get the LLM response from Groq
def get_llm_response(user_input):
  chat_completion = client.chat.completions.create( messages=[{"role": "user", "content": user_input}], model="llama3-8b-8192", temperature=0.7)
  # Replace with your desired model )
  return chat_completion.choices[0].message.content

In [12]:
# Function to convert text to speech using gTTS
def text_to_speech(text, output_audio="output_audio.mp3"):
  tts = gTTS(text)
  tts.save(output_audio)
  return output_audio

In [13]:
# Main chatbot function to handle audio input and output
def chatbot(audio):
  # Step 1: Transcribe the audio using Whisper
  result = model.transcribe(audio)
  user_text = result["text"]

  # Step 2: Get LLM response from Groq
  response_text = get_llm_response(user_text)

  # Step 3: Convert the response text to speech
  output_audio = text_to_speech(response_text)
  return response_text, output_audio

In [ ]:
# Gradio interface for real-time interaction
iface = gr.Interface( fn=chatbot, inputs=gr.Audio(type="filepath"),
                     # Input from mic
                      outputs=[gr.Textbox(), gr.Audio(type="filepath")],
                      # Output: response text and audio
                      live=True )

# Launch the Gradio app
iface.launch()